In [89]:
import pandas as pd 
pd.options.display.max_columns = 999

In [6]:
collision = pd.read_csv('Collision__arcgis_rest_services_SDOT_EXT_DSG_datasharing_MapServer_51_.csv', sep=',')
street = pd.read_csv('Street_Details__arcgis_rest_services_SDOT_EXT_DSG_datasharing_MapServer_81_.csv', sep=',')
traffic = pd.read_csv('Traffic_Flow_Counts.csv', sep=',')

In [26]:
traffic.sample(3)
street.sample(3)
collision.sample(3)

,OBJECTID,INCKEY,COLDETKEY,ADDRTYPE,COLLISIONTYPE,DISTANCE,EXCEPTRSNCODE,EXCEPTRSNDESC,FATALITIES,INATTENTIONIND,INCDATE,INCDTTM,INJURIES,INTKEY,JUNCTIONTYPE,LIGHTCOND,LOCATION,PEDCOUNT,PEDCYLCOUNT,PEDROWNOTGRNT,PERSONCOUNT,REPORTNO,ROADCOND,SDOT_COLCODE,SDOT_COLDESC,SDOTCOLNUM,SEGKEY,SERIOUSINJURIES,SEVERITYCODE,SEVERITYDESC,SPEEDING,ST_COLCODE,ST_COLDESC,STATUS,UNDERINFL,VEHCOUNT,WEATHER,WIDTH,Shape,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,SPDCASENO
5363,5364,119144,119144,Block,NaN,271.0,NaN,NaN,0,NaN,12/22/2011 12:00:00 AM +0000,12/22/2011,0,NaN,Mid-Block (not related to intersection),NaN,SW ADMIRAL WAY BETWEEN 65TH AVE SW AND DEAD END,0,0,NaN,0,C759706,NaN,0.0,NOT ENOUGH INFORMATION / NOT APPLICABLE,11356030.0,21732.0,0,0,Unknown,NaN,NaN,NaN,Unmatched,NaN,0,NaN,0,"(47.57597468900008, -122.41705476899995)",0,0,N,NaN
129292,129293,46178,46178,Block,Parked Car,156.5,NaN,NaN,0,NaN,12/09/2005 12:00:00 AM +0000,12/9/2005,0,NaN,Mid-Block (not related to intersection),Dark - Street Lights On,1ST AVE BETWEEN CLAY ST AND BROAD ST,0,0,NaN,2,2071913,Dry,13.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",5343044.0,1020.0,0,1,Property Damage Only Collision,NaN,32,One Parked - One Moving,Matched,0,2,Clear or Partly Cloudy,0,"(47.61663518900008, -122.35202667299995)",0,0,N,05-524797
11292,11293,126712,126712,Block,Parked Car,280.5,NaN,NaN,0,NaN,09/26/2011 12:00:00 AM +0000,9/26/2011 4:00:00 PM,0,NaN,Mid-Block (not related to intersection),Unknown,16TH AVE S BETWEEN S BAYVIEW ST AND S LANDER ST,0,0,NaN,2,3545618,Unknown,15.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE...",11269037.0,2295.0,0,1,Property Damage Only Collision,NaN,32,One Parked - One Moving,Matched,0,2,Unknown,0,"(47.58024082600008, -122.31210720599995)",0,0,N,11-325175


In [8]:
# UNITDESC in street
# LOCATION in collision

In [52]:
collision_street = pd.merge(left=collision, right=street, how='inner', left_on='LOCATION', right_on='UNITDESC')
print len(collision)
print len(street)
print len(collision_street)
print collision.shape #(186197, 43)
#collision_street.to_csv('COLLISION_STREET_New.csv', sep=',', index=False)

print collision_street.shape #(175473, 94)
print street.UNITDESC.nunique() # 23792 



186197
23805
175473
(186197, 43)
(175473, 94)
23792


In [53]:
combined = collision_street[['ADDRTYPE', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 
                              'LIGHTCOND', 'WEATHER', 'ARTDESCRIPT', 'SPEEDLIMIT',
                             'SEGDIR', 'ONEWAY', 'SEGLENGTH', 'SURFACEWIDTH',
                             'SURFACETYPE_1', 'SURFACEDATE_1', 'NATIONHWYSYS', 'STREETTYPE',
                             'SLOPE_PCT']]

combined.sample(3)


,ADDRTYPE,INCDATE,INCDTTM,JUNCTIONTYPE,LIGHTCOND,WEATHER,ARTDESCRIPT,SPEEDLIMIT,SEGDIR,ONEWAY,SEGLENGTH,SURFACEWIDTH,SURFACETYPE_1,SURFACEDATE_1,NATIONHWYSYS,STREETTYPE,SLOPE_PCT
20541,NaN,05/20/2015 12:00:00 AM +0000,5/20/2015 5:20:00 PM,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
83324,Block,04/08/2010 12:00:00 AM +0000,4/8/2010 1:11:00 PM,Mid-Block (but intersection related),Daylight,Clear or Partly Cloudy,Minor Arterial,30.0,N,N,336.0,48.0,AC/PCC,07/25/2013 12:00:00 AM +0000,N,Urban Village Neighborhood,5.0
61337,Block,12/25/2016 12:00:00 AM +0000,12/25/2016 2:00:00 AM,Mid-Block (not related to intersection),Dark - Street Lights On,Clear or Partly Cloudy,Principal Arterial,30.0,NW,Y,307.0,38.0,AC/PCC,07/25/2013 12:00:00 AM +0000,N,Urban Village Main,4.0


In [54]:
combined['Weekday'] = pd.to_datetime(combined['INCDATE'].apply(lambda x : x.split(' ')[0])).dt.dayofweek

/Users/rajatshah/virtualenvs/fbml/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
combined.sample(3)

,ADDRTYPE,INCDATE,INCDTTM,JUNCTIONTYPE,LIGHTCOND,WEATHER,ARTDESCRIPT,SPEEDLIMIT,SEGDIR,ONEWAY,SEGLENGTH,SURFACEWIDTH,SURFACETYPE_1,SURFACEDATE_1,NATIONHWYSYS,STREETTYPE,SLOPE_PCT,Weekday
95669,Block,05/08/2014 12:00:00 AM +0000,5/8/2014 12:02:00 PM,Mid-Block (but intersection related),Daylight,Raining,Principal Arterial,30.0,N,Y,256.0,40.0,AC/PCC,06/18/2014 12:00:00 AM +0000,Y,Urban Village Main,5.0,3
60670,Block,03/27/2013 12:00:00 AM +0000,3/27/2013 4:45:00 PM,Mid-Block (not related to intersection),Daylight,Overcast,Principal Arterial,30.0,N,Y,430.0,0.0,NaN,NaN,N,NaN,2.0,2
82005,Block,12/18/2008 12:00:00 AM +0000,12/18/2008 11:00:00 AM,Driveway Junction,Daylight,Snowing,Principal Arterial,30.0,E,N,350.0,54.0,AC/PCC,07/25/2013 12:00:00 AM +0000,Y,Urban Village Main,1.0,3


In [86]:
combined['LIGHTCOND'].unique()
combined = pd.concat([combined, pd.get_dummies(combined['LIGHTCOND'])], axis=1).drop(labels = ['Unknown', 'Other'], axis=1)
combined = pd.concat([combined, pd.get_dummies(combined['WEATHER'])], axis=1).drop(labels = ['Unknown', 'Other'], axis=1)
combined.sample(3)
#df = pd.concat([df, pd.get_dummies(df['LIGHTCOND'])], axis=1); df
#pd.get_dummies(combined['LIGHTCOND']).drop(labels = ['Unknown', 'Other'], axis=1).sample(4)

,ADDRTYPE,INCDATE,INCDTTM,JUNCTIONTYPE,LIGHTCOND,WEATHER,ARTDESCRIPT,SPEEDLIMIT,SEGDIR,ONEWAY,SEGLENGTH,SURFACEWIDTH,SURFACETYPE_1,SURFACEDATE_1,NATIONHWYSYS,STREETTYPE,SLOPE_PCT,Weekday,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dawn,Daylight,Dusk,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dawn,Daylight,Dusk,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dawn,Daylight,Dusk,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dawn,Daylight,Dusk,Blowing Sand or Dirt or Snow,Clear or Partly Cloudy,Fog/Smog/Smoke,Overcast,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dawn,Daylight,Dusk,Blowing Sand or Dirt or Snow,Clear or Partly Cloudy,Fog/Smog/Smoke,Overcast,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing
101952,Block,06/15/2009 12:00:00 AM +0000,6/15/2009 1:03:00 PM,Mid-Block (not related to intersection),Daylight,Clear or Partly Cloudy,Not Designated,0.0,N,N,342.0,25.0,PCC,10/22/2013 12:00:00 AM +0000,N,Urban Village Neighborhood Access,3.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
38731,NaN,04/15/2005 12:00:00 AM +0000,4/15/2005 6:30:00 PM,NaN,Dark - Street Lights Off,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
120508,Block,12/09/2007 12:00:00 AM +0000,12/9/2007,Driveway Junction,Daylight,Clear or Partly Cloudy,Principal Arterial,30.0,S,N,669.0,50.0,AC,07/25/2013 12:00:00 AM +0000,Y,Urban Center Connector,6.0,6,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [87]:
combined['STREETTYPE'].unique()

array(['Urban Village Neighborhood Access', nan, 'Urban Village Main',
       'Industrial Access', 'Urban Village Neighborhood',
       'Neighborhood Yield Street', 'Downtown', 'Urban Center Connector',
       'Neighborhood Corridor', 'Downtown Neighborhood',
       'Minor Industrial Access', 'Downtown Neighborhood Access', 'Alley'], dtype=object)